In [3]:
from dotenv import load_dotenv
import os
import mysql.connector

load_dotenv() 

True

In [4]:


# Establish a connection to the MySQL server
cnx = mysql.connector.connect(user=os.getenv('DB_USER'), password=os.getenv('DB_PASSWORD'),
                              host=os.getenv('DB_HOST'),
                              database=os.getenv('DB_DATABASE'))



In [ ]:
# Create a cursor object
cursor = cnx.cursor()

# Execute a SQL query to select the rows you want to update
cursor.execute("SELECT * FROM table_name")

# Fetch the rows
rows = cursor.fetchall()

# Loop through the rows and update each row
for row in rows:
    # Execute an UPDATE SQL query to update the row
    cursor.execute(f"UPDATE table_name SET column_name = 'new_value' WHERE id = {row[0]}")

# Commit the changes
cnx.commit()

# Close the cursor and connection
cursor.close()
cnx.close()

In [53]:
import os
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine

# Establish a connection to the MySQL server
cnx = mysql.connector.connect(user=os.getenv('DB_USER'), password=os.getenv('DB_PASSWORD'),
                              host=os.getenv('DB_HOST'),
                              database=os.getenv('DB_DATABASE'))

# Create a cursor object
cursor = cnx.cursor()

# Use pandas to execute a SQL query and download the data into a DataFrame
df_matched = pd.read_sql('''
                SELECT *
                FROM v_listings_v3_1
                WHERE (listed_at = CURDATE() - INTERVAL 5 DAY
                 OR underoffer_at = CURDATE() - INTERVAL 5 DAY
                 OR sstc_at = CURDATE() - INTERVAL 5 DAY
                 OR reduced_at = CURDATE() - INTERVAL 5 DAY) and address_id is not null and district = 'M16'
                ;''', con=cnx)

df_unmatched = pd.read_sql('''
                SELECT *
                FROM v_listings_v3_1
                WHERE (listed_at = CURDATE() - INTERVAL 1 DAY
                 OR underoffer_at = CURDATE() - INTERVAL 1 DAY
                 OR sstc_at = CURDATE() - INTERVAL 1 DAY
                 OR reduced_at = CURDATE() - INTERVAL 1 DAY) and address_id is null
                ;''', con=cnx)


# Manipulate the DataFrame as needed
# df['column_name'] = 'new_value'

# Close the cursor and connection
cursor.close()
cnx.close()


C:\Users\Jason McGinty\AppData\Local\Temp\ipykernel_1296\2097270803.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_matched = pd.read_sql('''
C:\Users\Jason McGinty\AppData\Local\Temp\ipykernel_1296\2097270803.py:24: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_unmatched = pd.read_sql('''


In [59]:
df_matched['listed_at'] = pd.to_datetime(df_matched['listed_at']).dt.date
df_matched['underoffer_at'] = pd.to_datetime(df_matched['underoffer_at']).dt.date
df_matched['sstc_at'] = pd.to_datetime(df_matched['sstc_at']).dt.date
try:
    df_matched['reduced_at'] = pd.to_datetime(df_matched['reduced_at'], errors = 'coerce').dt.date
except Exception as e:
    print(e)
    

In [63]:
print(df_matched['reduced_at'].isna().sum())

2


In [64]:
df_matched[['listed_at', 'underoffer_at', 'sstc_at', 'reduced_at']].head()

,listed_at,underoffer_at,sstc_at,reduced_at
0,2023-12-08,2024-01-12,2024-01-25,NaT
1,2024-01-25,NaT,NaT,NaT


In [61]:
df_matched.dtypes

id                         int64
address_id                 int64
status_id                  int64
type                      object
category                  object
subcategory               object
beds                       int64
baths                    float64
current_price              int64
original_price             int64
agency_id                  int64
agency_name               object
listed_at                 object
underoffer_at             object
sstc_at                   object
reduced_at        datetime64[ns]
street                    object
district                  object
sector                    object
postcode                  object
lat                      float64
lon                      float64
image                     object
thumbnail                 object
listing_url               object
dtype: object

In [62]:
df_matched['max_date'] = df_matched[['listed_at', 'underoffer_at', 'sstc_at', 'reduced_at']].apply(max, axis=1)
df_matched.head()

TypeError: Cannot compare NaT with datetime.date object

In [25]:
import requests
import json

for index,row in df_matched.iterrows():
    print(df_matched.loc[index,['address_id', 'status_id']])

address_id    6361810
status_id          12
Name: 0, dtype: object
address_id    6324897
status_id           1
Name: 1, dtype: object


In [34]:
def status_id_enum(id):
    if id == 1:
        return 'onmarket'
    elif id == 2:
        return 'underoffer'
    elif id == 12:
        return 'sstc'

In [26]:
def get_doc_id_ai_id(address_id):
    url = f"{os.getenv('ELASTIC_HOSE')}/agent_sample_unified_search_index/_search"

    # specify the query
    query = {
        "query": {
            "match": {
                "address_id": address_id
            }
        }
    }

    # send a POST request
    response = requests.post(url, headers={"Content-Type": "application/json"}, data=json.dumps(query))
    return json.loads(response.text)['hits']['hits'][0]['_id']

In [65]:
import datetime as dt

def insert_updated_details_os(row):
    doc_id = get_doc_id_ai_id(row['address_id'])
    # specify the url
    url = f"{os.getenv('ELASTIC_HOSE')}/agent_sample_unified_search_index/_update/{doc_id}"


    if row['type'] == 'S':
    # specify the update
        update = {
            "doc": {
                "sales_event_status": dt.date.today(),
                "sales_status_id": status_id_enum(row['status_id']),
                "latest_listing_price": row['current_price'],
                "listing_activity_status": True
            }
        }

    # send a POST request
    response = requests.post(url, headers={"Content-Type": "application/json"}, data=json.dumps(update))

    # print the response
    print(json.loads(response.text))

In [30]:
print(url)

NameError: name 'url' is not defined

In [66]:
for index,row in df_matched.iterrows():
    print(df_matched.loc[index,['address_id', 'status_id']])
    insert_updated_details_os(row)

address_id    6361810
status_id          12
Name: 0, dtype: object


AttributeError: module 'datetime' has no attribute 'today'

In [ ]:
# newly_added
df_matched = pd.read_sql('''
                SELECT *
                FROM v_listings_v3_1
                WHERE listed_at = CURDATE() - INTERVAL 1 DAY and address_id is not null
                ;''', con=cnx)